In [12]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from scipy.stats import norm
from utils.hedging import delta_hedge, run_delta_hedge_analysis

### Data structure
| index | date | C | K | expiration_date | initial_moneyness | option_id | S | r | TTM | delta | IV |

First, let's load the data and restrict to only 45 days before expiration.

In [13]:
data = pd.read_csv("../data/processed/TSLA_processed.csv")

data['date'] = pd.to_datetime(data['date'])
data['expiration_date'] = pd.to_datetime(data['expiration_date'])

# Not using TTM to avoid floating point issues
data = data[(data['expiration_date'] - data['date']).dt.days <= 45]

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/TSLA_processed.csv'

Now, validating expiration date/moneyness combinations.

In [ ]:
nan_counts = (
    data
    .assign(iv_is_nan = (data['IV'].isna() & (data['TTM'] != 0)).astype(int))
    .groupby(['expiration_date', 'initial_moneyness'])['iv_is_nan']
    .sum()
)
nan_table = nan_counts.unstack(fill_value=0)
nan_table

initial_moneyness,ATM,ITM,OTM
expiration_date,,,
2024-01-19,0,0,0
2024-02-16,0,0,0
2024-03-15,0,0,0
2024-04-19,0,0,0
2024-05-17,0,0,0
2024-06-21,0,1,0
2024-07-19,5,7,4
2024-08-16,0,0,0
2024-09-20,0,1,1


Let's construct data

In [ ]:
# def delta_hedge(data: pd.DataFrame, hedge_frequency: int = 1) -> None:
#     """
#     data: DataFrame
#         Contains data for a single option series
#     """

#     data = data.sort_values(by='date').reset_index(drop=True)
#     N = len(data)

#     delta = data.loc[0, 'delta']

#     errors = []

#     for i in range(1, N):
#         C1 = data.loc[i - 1, 'C']
#         C2 = data.loc[i, 'C']
#         S1 = data.loc[i - 1, 'S']
#         S2 = data.loc[i, 'S']

#         A = (C2 - C1) - delta * (S2 - S1)

#         errors.append(A)

#         if i % hedge_frequency == 0:
#             delta = data.loc[i, 'delta']
    
#     errors = np.array(errors)
#     mse = np.mean(errors**2)
    
#     result = {
#         'K' : data['K'].iloc[0],
#         'expiration_date' : data['expiration_date'].iloc[0],
#         'initial_moneyness' : data['initial_moneyness'].iloc[0],
#         'hedge_frequency' : hedge_frequency,
#         'mse' : mse,
#         'mean_error' : np.mean(errors),
#         'std_error' : np.std(errors)
#     }
#     return result

In [ ]:
# def run_delta_hedge_analysis(original_data: pd.DataFrame,
#                              hedge_frequency: int,
#                              moneyness: str,
#                              validation_table: pd.DataFrame) -> pd.DataFrame:
#     results = []

#     # choose only specified moneynesses and valid maturities
#     data = original_data[original_data['initial_moneyness'] == moneyness].copy()
    
#     flags = validation_table[moneyness]
#     mask = data['expiration_date'].map(flags) == 0
#     data = data[mask]

#     for _, opt_data in data.groupby('option_id'):
#         res = delta_hedge(data = opt_data,
#                           hedge_frequency = hedge_frequency)
#         if res is not None:
#             results.append(res)
#         else:
#             print(f"Skipping option_id {opt_data['option_id'].iloc[0]}.")
    
#     summary = pd.DataFrame(results)
#     return summary

First, let's try hedging all available ATM options.

In [ ]:
summary_daily = run_delta_hedge_analysis(data = data,
                                         frequencies = 1,
                                            moneyness = 'ATM',
                                            validation_table = nan_table)

In [ ]:
print(summary_daily)

       K expiration_date initial_moneyness  hedge_frequency       mse  \
0  240.0      2024-01-19               ATM                1  0.255771   
1  250.0      2024-02-16               ATM                1  0.137260   
2  190.0      2024-03-15               ATM                1  0.331747   
3  180.0      2024-04-19               ATM                1  0.199582   
4  165.0      2024-05-17               ATM                1  1.175112   
5  180.0      2024-06-21               ATM                1  0.317616   
6  230.0      2024-08-16               ATM                1  0.543789   
7  200.0      2024-09-20               ATM                1  0.707916   
8  210.0      2024-10-18               ATM                1  0.574114   
9  260.0      2024-11-15               ATM                1  3.515505   

   mean_error  std_error  
0   -0.148207   0.483534  
1    0.002108   0.370481  
2    0.031599   0.575107  
3   -0.055492   0.443286  
4    0.098002   1.079587  
5   -0.110827   0.552570  
6    0.